In [106]:
import pandas as pd                                     
import numpy as np                                      
import os 



import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline
import seaborn as sns                                   # For pretty plots
import gensim
import nltk


from os import path
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
from gensim import corpora, models, similarities
from nltk.corpus import stopwords

In [107]:
d = path.dirname("hillary-clinton-emails/")

# Read the whole text.
#emailJoined = open(path.join(d, 'Emails.csv')).read(200000)
emailJoined = open(path.join(d, 'Emails.csv')).read(1000)



[From Strings to vector](https://radimrehurek.com/gensim/tut1.html)

In [105]:
#stoplist = set('for a of the and to in - 1 2 3 4 5 6 7 8 9 a b c d e f g h i j k l m n o p q r s t u v w x y z é + " * ç % & / ()'.split())
#print(stoplist)
#documents = ["Human machine interface for lab abc computer applications"]
#type(documents.split())

documents = emailJoined.replace('.',',').replace('<',',').replace('>',',').replace(' ',',').replace(':',',').split(',')
#documents
                               


In [124]:
stoplist = set(STOPWORDS)

for i in range(1,100) :
    stoplist.add(i)

stoplist.add("no.")
stoplist.add("no")
stoplist.add("pm")
stoplist.add("am")
stoplist.add("doc")
stoplist.add("docx")
stoplist.add("pdf")
stoplist.add("re")
stoplist.add("fw")


#print(stoplist)


In [125]:
texts = [[word for word in document.lower().split() if word not in stoplist and len(word) > 1]
         for document in documents]

#print(texts)
#texts

In [126]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
     for token in text:
            frequency[token] += 1

texts = [[token for token in text if frequency[token] > 10]
          for text in texts]



For representing the questions only by ids (intergers) we need a dictonary to store the mapping.

In [127]:
dictionary = corpora.Dictionary(texts)
dictionary.save("part3/deerwester.dict")


In [128]:
print(dictionary)

Dictionary(15489 unique tokens: ['stupid', 'bipartisan', 'loretta', 'vietnamese', 'proper']...)


In [129]:
#print(dictionary.token2id)

--------------------

In [130]:
corpus = [dictionary.doc2bow(text) for text in texts]

#splited = emailJoined.split(',')
#corpus = [dictionary.doc2bow(text) for text in splited]




corpora.MmCorpus.serialize("part3/deerwester.mm", corpus)  # store to disk, for later use
#print(corpus)


--------------------

[Transformation interface](https://radimrehurek.com/gensim/tut2.html#available-transformations)

In [131]:
dictionary = corpora.Dictionary.load("part3/deerwester.dict")
corpus = corpora.MmCorpus('part3/deerwester.mm')

In [132]:
#tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

In [133]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=5)

In [134]:
print(model)

LdaModel(num_terms=15489, num_topics=5, decay=0.5, chunksize=2000)


In [135]:
model.print_topics(30)

[(0,
  '0.082*"08/31/2015" + 0.064*"date" + 0.045*"subject" + 0.019*"message" + 0.014*"huma" + 0.010*"one" + 0.010*"secretary" + 0.009*"time" + 0.009*"security" + 0.008*"states"'),
 (1,
  '0.017*"new" + 0.016*"com>" + 0.014*"government" + 0.013*"<hrod17@clintonemail" + 0.012*"gov\'" + 0.011*"dec" + 0.011*"also" + 0.011*"gov>"" + 0.010*"see" + 0.010*"pm""'),
 (2,
  '0.047*"sent" + 0.040*"release" + 0.028*"will" + 0.026*"""" + 0.023*"part" + 0.020*"gov>" + 0.019*""unclassified" + 0.017*"said" + 0.015*"original" + 0.013*"sullivan"'),
 (3,
  '0.093*"f-2014-20439" + 0.080*"00" + 0.068*"state" + 0.062*"department" + 0.040*"unclassified" + 0.037*"00+00" + 0.027*"december" + 0.020*"cheryl" + 0.019*"full" + 0.013*"jacob"'),
 (4,
  '0.068*"2010" + 0.062*"case" + 0.020*"clinton_email_august_release" + 0.020*"2015-08-31t04" + 0.016*"b6" + 0.013*"10" + 0.010*"15" + 0.009*"people" + 0.008*"start" + 0.007*"14"')]

--------------------------

In [123]:
emails = pd.read_csv("hillary-clinton-emails/Emails.csv")
emails.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [3]:
edges = emails[['SenderPersonId', 'MetadataTo', 'ExtractedBodyText']]
edges = edges.dropna(subset=['SenderPersonId', 'MetadataTo'])
edges['MetadataTo'] = edges['MetadataTo'].apply(lambda x: x.lower())
edges.shape

(7665, 3)

In [6]:
aliases = pd.read_csv("hillary-clinton-emails/Aliases.csv")
persons = pd.read_csv("hillary-clinton-emails/Persons.csv")
edgesId = pd.merge(edges, aliases[['Alias', 'PersonId']], how='inner', left_on='MetadataTo', right_on='Alias')
edgesId = edgesId.drop(['MetadataTo','Alias'], axis=1)
print("merge 1:", edgesId.shape)
edgesId = pd.merge(edgesId, persons, how='inner', left_on='SenderPersonId', right_on='Id')
edgesId = edgesId.drop(['SenderPersonId','Id'], axis=1)
edgesId.columns = ['ExtractedBodyText', 'PersonId', 'Sender']
print("merge 2:", edgesId.shape)
edgesId = pd.merge(edgesId, persons, how='inner', left_on='PersonId', right_on='Id')
edgesId = edgesId.drop(['PersonId','Id'], axis=1)
edgesId.columns = ['ExtractedBodyText', 'Sender', 'Recipient']
print("merge 3:", edgesId.shape)
edgesId.head()
edgesId.head()

merge 1: (7392, 3)
merge 2: (7392, 3)
merge 3: (7392, 3)


,ExtractedBodyText,Sender,Recipient
0,NaN,Jake Sullivan,Hillary Clinton
1,FYI,Jake Sullivan,Hillary Clinton
2,Fyi\nB6\n— —,Jake Sullivan,Hillary Clinton
3,Fyi,Jake Sullivan,Hillary Clinton
4,This is nice.,Jake Sullivan,Hillary Clinton


In [65]:
edgesId.head()

,ExtractedBodyText,Sender,Recipient
0,NaN,Jake Sullivan,Hillary Clinton
1,FYI,Jake Sullivan,Hillary Clinton
2,Fyi\nB6\n— —,Jake Sullivan,Hillary Clinton
3,Fyi,Jake Sullivan,Hillary Clinton
4,This is nice.,Jake Sullivan,Hillary Clinton


In [79]:
edgesId.to_csv("part3/edgesId.csv")

In [124]:
with open("part3/edgesId.csv") as data : 
    emailJoined = data.read(1000)

In [125]:
#emailJoined
documents = emailJoined.replace('.',',').replace('<',',').replace('>',',').replace(' ',',').replace(':',',').split(',')


In [126]:
#stoplist = set(STOPWORDS)
stoplist = stopwords.words('english')

'''for i in range(1,100) :
    stoplist.append(i)'''
'''
stoplist.add("no.")
stoplist.add("no")
stoplist.add("pm")
stoplist.add("am")
stoplist.add("doc")
stoplist.add("docx")
stoplist.add("pdf")
stoplist.add("re")
stoplist.add("fw")
stoplist.add("-")'''


#print(stoplist)

print(stoplist)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [127]:
texts = [[word for word in document.lower().split() if word not in stoplist and len(word) > 1]
         for document in documents]

#print(texts)
#texts

In [128]:
#print(texts)

In [129]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
     for token in text:
            frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
          for text in texts]


In [130]:
dictionary = corpora.Dictionary(texts)
dictionary.save("part3/deerwester.dict")

In [115]:
print(dictionary)

Dictionary(18535 unique tokens: ['euna', 'national', 'lasted', 'efforts', 'group*']...)


In [131]:
print(texts)

[[], [], [], [], [], ['jake'], ['sullivan'], ['hillary'], ['clinton'], ['fyi'], ['jake'], ['sullivan'], ['hillary'], ['clinton'], ['"fyi'], ['—"'], ['jake'], ['sullivan'], ['hillary'], ['clinton'], ['fyi'], ['jake'], ['sullivan'], ['hillary'], ['clinton'], [], [], [], [], ['jake'], ['sullivan'], ['hillary'], ['clinton'], [], ['jake'], ['sullivan'], ['hillary'], ['clinton'], [], [], [], ['jake'], ['sullivan'], ['hillary'], ['clinton'], ['fyi'], ['jake'], ['sullivan'], ['hillary'], ['clinton'], ['fyi'], ['jake'], ['sullivan'], ['hillary'], ['clinton'], ['"fyi'], ['jake'], ['sullivan'], ['hillary'], ['clinton'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['—"'], ['jake'], ['sullivan'], ['hillary'], ['clinton'], [], [], ['sullivan'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['jake'], ['sullivan'], ['hillary'], ['clinton'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['course'], [], [], [], [], [

In [116]:
#print(dictionary.token2id)

In [117]:
corpus = [dictionary.doc2bow(text) for text in texts]

#splited = emailJoined.split(',')
#corpus = [dictionary.doc2bow(text) for text in splited]




corpora.MmCorpus.serialize("part3/deerwester.mm", corpus)  # store to disk, for later use
#print(corpus)


In [118]:
dictionary = corpora.Dictionary.load("part3/deerwester.dict")
corpus = corpora.MmCorpus('part3/deerwester.mm')

In [119]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=5)

In [120]:
print(model)

LdaModel(num_terms=18535, num_topics=5, decay=0.5, chunksize=2000)


In [121]:
model.print_topics(30)

[(0,
  '0.208*"clinton" + 0.019*"valmoro" + 0.015*"time" + 0.014*"2010" + 0.013*"us" + 0.013*"gov\'" + 0.011*"want" + 0.010*"best" + 0.009*"help" + 0.009*"10"'),
 (1,
  '0.207*"hillary" + 0.020*"would" + 0.019*"know" + 0.017*"could" + 0.015*""h" + 0.011*"need" + 0.009*"pis" + 0.008*"go" + 0.008*"blumenthal" + 0.008*"office"'),
 (2,
  '0.037*"huma" + 0.026*"jiloty" + 0.023*"cheryl" + 0.018*"work" + 0.018*"see" + 0.018*"also" + 0.015*"people" + 0.014*"un" + 0.010*"settlements" + 0.010*"new"'),
 (3,
  '0.035*"abedin" + 0.023*"call" + 0.022*"pm" + 0.022*"mills" + 0.021*"com" + 0.018*"get" + 0.016*"i\'m" + 0.015*"print" + 0.014*"like" + 0.013*"ok"'),
 (4,
  '0.028*"lauren" + 0.028*"jake" + 0.027*"sullivan" + 0.023*"lona" + 0.021*"thx" + 0.020*"pls" + 0.020*"hrod17@clintonemail" + 0.010*"haitian" + 0.010*"tomorrow" + 0.009*"may"')]

In [132]:
print(corpus)

MmCorpus(623469 documents, 18535 features, 346146 non-zero entries)
